In [73]:
import pandas as pd
import numpy as np

In [74]:
df=pd.read_csv('../data/processed_train_filled.csv')

In [75]:
edu_df = df['education'].map({'illiterate':0,'basic.4y':1,'basic.6y':2,'basic.9y':3,
                                  'high.school':4,'professional.course':5,'university.degree':6})
job_df = df['job'].map({'unemployed': 0,'student': 0,'retired': 0,'housemaid': 0,'entrepreneur': 1,
                              'self-employed': 1,'blue-collar': 2,'services': 3,'technician': 4,'admin.': 5,'management': 5})
def_df = df['default'].map({'yes':1,'no':0})
housing_df = df['housing'].map({'yes':1,'no':0})
loan_df = df['loan'].map({'yes':1,'no':0})
debt_df = pd.DataFrame(columns=['debt'])
debt_df['debt'] = def_df + housing_df + loan_df
train_df = pd.concat([edu_df, job_df, debt_df, df[['age', 'campaign', 'pdays', 'previous', 'duration']]], axis=1)
train_df

,education,job,debt,age,campaign,pdays,previous,duration
0,5,5,2,51,1,112,2,4621
1,4,3,1,50,1,412,2,4715
2,3,2,0,48,0,1027,1,171
3,4,1,3,26,26,998,0,359
4,6,5,0,45,1,240,4,3178
...,...,...,...,...,...,...,...,...
22495,4,5,0,33,3,925,3,4031
22496,4,5,1,34,14,533,3,3185
22497,5,5,1,25,0,792,5,2363
22498,4,0,1,57,5,989,0,576


In [76]:
mon_df = df['month'].map({'mar':3, 'apr':4, 'may':5, 'jun':6, 'jul':7, 'aug':8, 'sep':9, 'oct':10, 'nov':11, 'dec':12})
day_df = df['day_of_week'].map({'mon':1, 'tue':2, 'wed':3, 'thu':4, 'fri':5})
train_df = pd.concat([train_df, mon_df, day_df], axis=1)

In [77]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
numeric_X = pd.DataFrame(scaler.fit_transform(train_df), columns = train_df.columns)

In [78]:
y = df['subscribe'].map({'yes':1,'no':0})

In [79]:
# import train_test split
from sklearn.model_selection import train_test_split
# split data into train and test, 80% in training and 20% in testing
X_train, X_test, y_train, y_test = train_test_split(numeric_X, y, test_size=0.2,
                                                    random_state = 99)

In [80]:
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

X_with_intercept = sm.add_constant(X_train)
stats_model = sm.OLS(y_train, X_with_intercept)
results = stats_model.fit()

print("P-values:")
print(results.pvalues)


P-values:
const          0.000000e+00
education      7.398865e-15
job            4.736404e-10
debt           2.535996e-04
age            1.603796e-12
campaign       4.565386e-74
pdays          9.350059e-35
previous       1.233935e-02
duration       2.032486e-01
month          4.585310e-07
day_of_week    1.024654e-01
dtype: float64


In [81]:
coeff_df = pd.DataFrame(lin_reg.coef_, numeric_X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
education,0.021136
job,-0.016847
debt,0.009251
age,0.018316
campaign,0.047631
pdays,-0.039107
previous,-0.007994
duration,-0.003957
month,0.012616
day_of_week,0.004034
